## Loading of modules and packages

Activate the Julia environment of the project.

In [ ]:
using Pkg
Pkg.activate("..\\")

Load the necessary packages.

In [ ]:
using ArchGDAL

Load the necessary modules.

In [ ]:
include(".\\Data Gathering\\GroundData.jl")
include(".\\Data Gathering\\SatelliteData.jl")
include(".\\Analysis\\DiluitionAttenuationFactor.jl")
include(".\\Analysis\\Lakes.jl")
include(".\\Analysis\\Noises.jl")
include(".\\Analysis\\Plumes.jl")
include(".\\Analysis\\Sediments.jl")


Aliasing of the modules to shorten the code.

In [ ]:
# Modules
const agd = ArchGDAL

# Data gathering
const grdt = GroundData
const stdt = SatelliteData

# Data Analysis
const daf = DiluitionAttenuationFactor 
const lakes = Lakes
const noises = Noises
const plumes = Plumes
const sediments = Sediments

## Code testing 

#### Data Gathering testing

In [ ]:
grdt.getGroundData( :METEO, grdt.AA, grdt.FVG, grdt.L, grdt.T, grdt.V )

#### Data Analysis testing

##### Initial Setup

In [ ]:
# Digital Terrain Model, the main raster for the analysis, reppresents Veneto
dtm_file = "..\\resources\\Analysis data\\DTM_32.tiff"
dtm = agd.read(dtm_file)

Create a source point to test the analysis functions.

In [ ]:
# Coordinates of the source
# lat, lon = (726454.9302346368, 5.025993899219433e6)
lat, lon = (11.930065824163105, 45.425861311724816) # WGS84
# Path to the source shapefile
source_dir = "..\\resources\\Analysis data\\source_shapefile"
# Directory holding the `source` files
!isdir(source_dir) && mkdir(source_dir)
# Creation of a shapefile containing the point if not already existing
if "source.shp" ∉ readdir(source_dir)
    agd.create(
        source_dir*"\\source.shp",
        driver = agd.getdriver("ESRI Shapefile")
    ) do ds
        agd.createlayer(
            geom = agd.wkbPoint,
            spatialref = agd.importEPSG(4326)
        ) do layer
            agd.createfeature(layer) do feature
                agd.setgeom!(feature, agd.createpoint(lat, lon))
            end
            agd.copy(layer, dataset=ds)
        end
    end
end
source_file = source_dir*"\\source.shp"
source = agd.read(source_file)

Analysis functions execution.

Pollutants leaching.

In [ ]:
#=
contaminants: (vettore di String)
    nomi della sostanze da usare per estrarre i valori "c_henry", "koc_kd" dal database.
concentrations: (vettore di Float64)
    concentrazioni delle sostanze.
aquifer_depth: (Float64)
    profondità della falda.
acquifer_flow_direction: (Int64)
    angolo del flusso dell'aqua nella falda in gradi (credo?)
mean_rainfall: (Float64)
    pioggia media
texture: (String)
    credo sia la tipologia di terreno in cui filtrano gli inquinanti, da usare per estrarre i valori "tot_por", "c_water_avg", "effective_infiltration",
    "por_eff", "grain" dal database.
orthogonal_extension: (Float64)
    estensione ortogonale?
soil_density: (Float64)
    densità del suolo
source_thickness: (Float64)
    spessore fonte?
darcy_velocity: (Float64)
    velocità di Darcy?
mixed_zone_depth: (Float64)
    profondità zoma mista?
decay_coeff: (Float64)
    coefficiente di decadimento?
=#
daf.run_leach(source_file, contaminants, concentrations, aquifer_depth,
    acquifer_flow_direction, mean_rainfall, texture, 25, time,
    orthogonal_extension, soil_density, source_thickness, darcy_velocity,
    mixed_zone_depth, decay_coeff, algorithm,
    "..\\resources\\Analysis test data\\Analysis results\\daf.tiff")

Polutants dispersion in lakes.

In [ ]:
#=
wind_direction: (Int64)
    direzione del vento come angolo in gradi.
pollutant_mass: (Float64)
    massa dell'inquinante.
flow_mean_speed: (Float64)
    velocità del flusso.
hours: (Int64)
    durata dell'analisi in ore.
fickian_x: (Float64)
    ?
fickian_y: (Float64)
    ?
λk: (Float64)
    ?
=#
lakes.run_lake(source_file, wind_direction, pollutant_mass, flow_mean_speed, 25,
    hours, fickian_x, fickian_y, λk,
    "..\\resources\\Analysis test data\\Analysis results\\lake.tiff")

Noise pollution

In [ ]:
#=
"A" sostituisce il raster delle impedenze (nel codice uso una matrice contenente solo 0)
temperatura atmosferica in kelvin
umidità relativa (credo come percentuale in decimale)
intensità del suono
frequenza del suono
=#
noises.run_noise( dtm_file, "A", source_file, 293.15, 0.2, 110.0, 400.0 )

Airborne pollutants dispersion from a stack.

In [ ]:
#=
stability: (String)
    classe di stabilità da usare per ottenere     dal database.
wind_direction: (Int64)
    direzione del vento come angolo.
concentration: (Float64)
    concentrazione della sostanza.
wind_speed: (Float64)
    velocità del vento.
stack_height: (Float64)
    altezza del camino.
gas_speed: (Float64)
    velocità dei fumi?
stack_diameter: (Float64)
    diametro del camino.
smoke_temperature: (Float64)
    temperatura dei fumi.
temperature: (Float64)
    temperatura dell'ambiente.
=#
plumes.run_plume(dtm_file, source_file, stability, 25, wind_direction, concentration,
    wind_speed, stack_height, gas_speed, stack_diameter, smoke_temperature,
    temperature,
    "..\\resources\\Analysis test data\\Analysis results\\plumes.tiff")

Pollutants dispersion in flowing water.

In [ ]:
#=
mean_flow_speed: (Float64)
    velocità media della corrente.
mean_depth: (Float64)
    profondità media.
x_dispersion_coeff: (Float64)
    coefficiente di dispersione su x?
y_dispersion_coeff: (Float64)
    coefficiente di dispersione su y?
dredged_mass: (Float64)
    massa trasportata.
flow_direction: (Int64)
    direzione del flusso come angolo in gradi.
mean_sedimentation_velocity: (Float64)
    velocità di sedimentazione.
time: (Int64)
    tempo di inizio del modello.
time_intreval: (Int64)
    lunghezza di un'epoca.
current_oscillatory_amplitude: (Float64)
    ampiezza di oscillazione della corrente?
tide: (Int64)
    ciclo di marea in ore?.
=#
sediments.run_sediment(dtm_file, source_file, 25, mean_flow_speed, mean_depth,
    x_dispersion_coeff, y_dispersion_coeff, dredged_mass, flow_direction,
    mean_sedimentation_velocity, time, time_intreval,
    current_oscillatory_amplitude, tide,
    "..\\resources\\Analysis test data\\Analysis results\\sediments.tiff")